adapted from :https://medium.com/@chand.shelvin/pybullet-getting-started-a068a0e3d492

http://www.mymodelrobot.appspot.com/5674976526991360

https://wiki.ros.org/urdf/XML/link

https://www.programcreek.com/python/example/122099/pybullet.setJointMotorControl2

In [27]:
import pybullet as p
import time
import pybullet_data

In [26]:
import random as r
from urdfpy import URDF
import urdfpy
import numpy as np

In [30]:
physicsClient = p.connect(p.GUI)# p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath()) # optionally
p.setGravity(0,0,-9.8)   
planeId = p.loadURDF('plane.urdf')
cubeStartPos = [0,0,1]
cubeStartOrientation = p.getQuaternionFromEuler([0,0,0])
boxId = p.loadURDF('Models/Model_0.urdf',cubeStartPos, cubeStartOrientation)
p.setJointMotorControl2(boxId,0,p.POSITION_CONTROL,1)
p.setJointMotorControl2(boxId,1,p.POSITION_CONTROL,-1)
p.setJointMotorControl2(boxId,2,p.POSITION_CONTROL,1)
cubePos, cubeOrn = p.getBasePositionAndOrientation(boxId)
print("start")
print(cubePos,cubeOrn)
p.resetDebugVisualizerCamera( cameraDistance=3.5, cameraYaw=30, cameraPitch=52, cameraTargetPosition=cubePos)
for i in range (300):
   p.stepSimulation()
   time.sleep(1./200.)
cubePos, cubeOrn = p.getBasePositionAndOrientation(boxId)
print(cubePos,cubeOrn)
 

start
(0.0, 0.0, 1.0) (0.0, 0.0, 0.0, 1.0)
(-0.11113194030165471, -0.008329735254868794, 0.2174921530692237) (0.008226794146440262, -0.10610485923263989, 0.07686680089920954, 0.9913453352010162)


In [31]:
p.disconnect()

In [11]:
def create_base_link(xyz_rpy):
    col = urdfpy.Collision(name=None,origin=urdfpy.xyz_rpy_to_matrix(xyz_rpy),geometry=urdfpy.Geometry(urdfpy.Box([.67,.67,.3]),None,None,None))
    vis = urdfpy.Visual(urdfpy.Geometry(urdfpy.Box([.67,.67,.3]),None,None,None))    
    return urdfpy.Link("base_link",collisions=[col],inertial=None,visuals=[vis])

def create_arm_link(name,xyz_rpy):
    col = urdfpy.Collision(name=None,origin=urdfpy.xyz_rpy_to_matrix(xyz_rpy),geometry=urdfpy.Geometry(urdfpy.Box([.67,.67,.3]),None,None,None))
    vis = urdfpy.Visual(urdfpy.Geometry(urdfpy.Box([.67,.67,.3]),None,None,None),origin=urdfpy.xyz_rpy_to_matrix(xyz_rpy))    
    return urdfpy.Link(name,collisions=[col],inertial=urdfpy.Inertial(1,[[0, 0, 0],[0, 0, 0],[0, 0, 0]],urdfpy.xyz_rpy_to_matrix(xyz_rpy)),visuals=[vis])

def create_joint(parent,child,xyz_rpy,name):
    child.origin = xyz_rpy
    return urdfpy.Joint(name,joint_type="revolute",parent=parent.name,child=child.name,limit=urdfpy.JointLimit(1,1,-1,1),origin=xyz_rpy)


In [ ]:
arm1 = create_arm_link("test",[0,.34,0,0,0,0])
base = create_base_link([0,0,0,0,0,0])


# robot = URDF("test",links=[base,arm1], joints=[joint1])

# robot.save("robotTest.urdf")

In [25]:
def generateModels(amountOfModelsToBeGenerated, amountOfMotorOnEachModel):
    for i in range(0,amountOfModelsToBeGenerated):
        jointList = []
        linksList = []
        base = create_base_link([0,0,0,0,0,0])
        linksList.append(base)
        for j in range(amountOfMotorOnEachModel):
            x =r.uniform(-.7,.7) 
            y = r.uniform(-.7,.7)
            z = r.uniform(-.1,.1)
            pitch = r.uniform(-1,1) 
            yaw = r.uniform(-1,1) 
            roll = r.uniform(-1,1) 
            arm1 = create_arm_link("arm"+str(j),[x,y,z,pitch,yaw,roll])
            linksList.append(arm1)
            jointList.append(create_joint(base,arm1,[x,y,z,pitch,yaw,roll],str(j)))
        robot = URDF("Model_"+str(i),links=linksList, joints=jointList)
        robot.save("Models/Model_"+str(i)+".urdf")
generateModels(2,3)

https://urdfpy.readthedocs.io/en/latest/generated/urdfpy.Link.html#urdfpy.Link

https://urdfpy.readthedocs.io/en/latest/examples/index.html#loading-from-a-file
